In [1]:
import pandas as pd

pd.set_option("display.precision", 3)
import os
import warnings

warnings.filterwarnings("ignore")
from sklearn.cluster import SpectralClustering
from mvlearn.cluster import MultiviewKMeans
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from IPython.display import display
import sys
import math
import seaborn as sns

sns.set_style("white")
from ConsensusClusteringMultiView import ConsensusCluster
import scipy.stats as sps
import copy
from tqdm import tqdm
from sklearn.manifold import TSNE
import matplotlib.cm as cm
import matplotlib.lines as mlines
from sklearn.cluster import DBSCAN
from pathlib import Path
import scipy.stats as stats



In [2]:
def cal_F_stat(data):
    SSB = (
        ((data.groupby("cluster").mean() - data.drop("cluster", 1).mean()) ** 2).T
        * data.groupby("cluster").size().values
    ).sum(1)

    SSW = []
    for k in sorted(data["cluster"].unique()):
        cluster = data[data["cluster"] == k].drop("cluster", 1)
        diff = (cluster - data.groupby("cluster").mean().loc[k]) ** 2
        SSW.append(diff)
    SSW = pd.concat(SSW).sum()
    coef = (len(data) - len(data["cluster"].unique())) / (
        len(data["cluster"].unique()) - 1
    )
    F_stat = (coef * (SSB / SSW)).sort_values(ascending=False)
    return F_stat

def plot_top(data, target_var, save_path, name):
    plt.figure(figsize=(30, 10))
    target_cluster = np.unique(data["cluster"])
    for i, var in enumerate(target_var):
        plt.subplot(2, 5, i + 1)
        for k in target_cluster:
            cluster = data[data["cluster"] == k]
            plt.hist(
                cluster[var],
                bins=25,
                alpha=0.5,
                density=True,
                histtype="stepfilled",
                label="cluster {}, size {}".format(k, len(cluster)),
            )
        plt.legend()
        plt.title(var, fontsize=15)
    plt.suptitle(name, fontsize=25)
   
    plt.savefig(
        "{}/{}.png".format(save_path, name), dpi=300,
    )
    plt.close()
    
def get_pw_F_stat(data, num_var):
    F_stat_true = cal_F_stat(data)
    all_F_stat = []
    n_test = 19 * num_var
    
    for i in tqdm(range(n_test)):
        data_run = data.copy()
        rs = np.random.RandomState(i)
        data_run["cluster"] = rs.randint(1, len(data['cluster'].unique()) + 1, size=len(data_run))
        F_stat = cal_F_stat(data_run)
        all_F_stat.append(F_stat)
    
    all_F_stat = pd.concat(all_F_stat, 1)
    all_F_stat.columns = ["random {}".format(i + 1) for i in range(all_F_stat.shape[1])]
    all_F_stat["true"] = F_stat_true
    return all_F_stat

In [3]:
data_path = "data/"
score_path = "{}/Clustering_silhouette/".format(data_path)
cdf_path = "{}/CDF plots/".format(data_path)
tsne_path = "{}/TSNEplots/".format(data_path)
KCC_path = "{}/KCC/".format(data_path)
f_stat_path = '{}/F_stat/'.format(data_path)
top_feature_plot_path = '{}/Top_features/'.format(data_path)

In [4]:
proteome_view = pd.read_csv(
    "{}/ProteomeViewStandardized.csv".format(data_path), index_col=0
)
contextual_view = pd.read_csv(
    "{}/ContextualViewStandardized.csv".format(data_path), index_col=0
)
physio_view = pd.read_csv(
    "{}/PhysioViewStandardized.csv".format(data_path), index_col=0
)
clinical_binary = pd.read_csv("{}/ClinicalBinary.csv".format(data_path), index_col=0)
npx = pd.read_csv("../olinks/20191053_Giannoni_NPX.csv")
npx.set_index("Panel", inplace=True)
proteome_view = pd.read_csv(
    "{}/ProteomeViewStandardized.csv".format(data_path), index_col=0
)

uniport = {}
for k, p in enumerate(npx.columns):
    uniport[p] = npx.iloc[:, k]["Uniprot ID"]
uniprot_col = []
for col in proteome_view.columns:
    uniprot_col.append(uniport[col])

proteome_view.columns = uniprot_col

concated_view = pd.concat([physio_view, contextual_view, proteome_view], 1)
all_features = pd.concat([concated_view,clinical_binary ], 1)

In [5]:
nb_continous_vars = concated_view.shape[1]
nb_binary_vars = clinical_binary.shape[1]
total_vars = nb_continous_vars + nb_binary_vars 


In [6]:
configs = [
    ["clinical", 4, "ConsensusKMeans"],
    ["contextual", 5, "ConsensusKMeans"],
    ["physio", 3, "DBSCAN"],
    ["proteome", 3, "ConsensusKMeans"],
    ["proteome", 4, "ConsensusKMeans"],
]


# pair-wise Fisher exact

In [11]:
for i in range(len(configs)):
    view, KCC_space, method = configs[i]

    assignment = pd.read_csv(
        "{}/{}_{}_view_KCC_{}_assignments.csv".format(
            score_path, method, view, KCC_space
        ),
        index_col=0,
    )
    print(assignment.head())
    if method == "DBSCAN":
        assignment["assignment"] = assignment["assignment"] + 1
        assignment = assignment[assignment["assignment"] != 0]
    assignment["assignment"] = assignment["assignment"].astype(int)
    data = clinical_binary.copy()
    data["cluster"] = assignment["assignment"]
    data = data[data["cluster"].notnull()]
    
    num_cluster = len(data["cluster"].unique())
    all_clusters = sorted(data["cluster"].unique())

    for i in all_clusters:
        for j in all_clusters:
            if j > i:
                i, j = int(i), int(j)
                all_p_val = pd.Series(index=clinical_binary.columns)
                cluster_pw = data[(data["cluster"] == i) | (data["cluster"] == j)]
                save_name = "{}_{}_KCC_{}_FisherExact_cluster_{}vs{}.csv".format(
                    method, view, KCC_space, i, j
                )

                for col in clinical_binary.columns:
                    try:
                        contingency_table = pd.crosstab(
                            cluster_pw[col], cluster_pw["cluster"]
                        )
                        p_val = stats.fisher_exact(contingency_table)[1]
                        all_p_val.loc[col] = p_val
                    except:
                        pass
                all_p_val.to_csv("{}/{}".format(f_stat_path, save_name))

           assignment
sample.id            
BE-001              3
BE-003              4
BE-004              4
BE-005              2
BE-007              1
(387, 83)
           assignment
sample.id            
BE-001              5
BE-003              4
BE-004              2
BE-005              2
BE-007              3
(387, 83)
           assignment
sample.id            
BE-001              0
BE-003              1
BE-004              2
BE-005              1
BE-007              1
(368, 83)
           assignment
sample.id            
BE-001              1
BE-003              2
BE-004              1
BE-005              2
BE-007              1
(387, 83)
           assignment
sample.id            
BE-001              1
BE-003              2
BE-004              4
BE-005              2
BE-007              4
(387, 83)


# pair-wise F-stat

In [12]:
for i in range(len(configs)):
    view, KCC_space, method = configs[i]
    assignment = pd.read_csv(
        "{}/{}_{}_view_KCC_{}_assignments.csv".format(
            score_path, method, view, KCC_space
        ),
        index_col=0,
    )
    
    if method == "DBSCAN":
        assignment["assignment"] = assignment["assignment"] + 1
        assignment = assignment[assignment["assignment"] != 0]

    data = concated_view.copy()
    data["cluster"] = assignment["assignment"]
    data = data[data["cluster"].notnull()]
    print(data.shape)
    num_cluster = len(data["cluster"].unique())
    all_clusters = sorted(data["cluster"].unique())

    for i in all_clusters:
        for j in all_clusters:
            if j > i:
                i, j = int(i), int(j)
                cluster_pw = data[(data["cluster"] == i) | (data["cluster"] == j)]
                save_name = "{}_{}_KCC_{}_F_stat_cluster_{}vs{}.csv".format(
                    method, view, KCC_space, i, j
                )

                all_F_stat = get_pw_F_stat(cluster_pw, total_vars)
                F_stat_pw = all_F_stat["true"]
                F_stat_pw_pvalue = (
                    all_F_stat.rank(axis=1, ascending=False)["true"]
                    / all_F_stat.shape[1]
                )
                F_stat_pw = pd.concat([F_stat_pw, F_stat_pw_pvalue], 1)
                F_stat_pw.columns = ["F_stat", "p-value"]

                F_stat_pw.to_csv("{}/{}".format(f_stat_path, save_name))

(387, 465)


100%|████████████████████████████████████| 10374/10374 [01:09<00:00, 149.29it/s]


(387, 465)


100%|████████████████████████████████████| 10374/10374 [01:13<00:00, 141.76it/s]


(368, 465)


100%|████████████████████████████████████| 10374/10374 [01:11<00:00, 144.63it/s]


(387, 465)


100%|████████████████████████████████████| 10374/10374 [01:17<00:00, 134.51it/s]


(387, 465)


100%|████████████████████████████████████| 10374/10374 [01:13<00:00, 141.47it/s]


# plot top features

In [15]:
for i in range(len(configs)):
    view, KCC_space, method = configs[i]

    assignment = pd.read_csv(
        "{}/{}_{}_view_KCC_{}_assignments.csv".format(
            score_path, method, view, KCC_space
        ),
        index_col=0,
    )
    if method == "DBSCAN":
        assignment["assignment"] = assignment["assignment"] + 1
        assignment = assignment[assignment["assignment"] != 0]
    assignment["assignment"] = assignment["assignment"].astype(int)
    data = all_features.copy()
    data["cluster"] = assignment["assignment"]
    data = data[data["cluster"].notnull()]
    print(data.shape)
    num_cluster = len(data["cluster"].unique())
    all_clusters = sorted(data["cluster"].unique())

    for i in all_clusters:
        for j in all_clusters:
            if j > i:
                i, j = int(i), int(j)
                cluster_pw = data[(data["cluster"] == i) | (data["cluster"] == j)]
                fstat_name = "{}_{}_KCC_{}_F_stat_cluster_{}vs{}.csv".format(
                    method, view, KCC_space, i, j
                )
                F_stat = pd.read_csv(
                    "{}/{}".format(f_stat_path, fstat_name), index_col=0
                )

                fisher_name = "{}_{}_KCC_{}_FisherExact_cluster_{}vs{}.csv".format(
                    method, view, KCC_space, i, j
                )
                Fisher = pd.read_csv(
                    "{}/{}".format(f_stat_path, fisher_name), index_col=0
                )
                Fisher.columns = ["p-value"]
                Fisher["F_stat"] = np.nan

                F_stat = F_stat[F_stat["p-value"] < 0.05]
                Fisher = Fisher[Fisher["p-value"] < 0.05 /total_vars]
                stats = pd.concat([F_stat, Fisher])
                stats = (
                    stats.sort_values("p-value")
                    .sort_values("F_stat", ascending=False)
                    .iloc[:10]
                )
                fig_name = "{}_{}_KCC_{}_TopFeatures_cluster_{}vs{}".format(
                    method, view, KCC_space, i, j
                )
                plot_top(
                    cluster_pw, stats.index.tolist(), top_feature_plot_path, fig_name
                )

(387, 547)
(387, 547)
(368, 547)
(387, 547)
(387, 547)
